

# $\color{blue}{RAGAS}$

---

There are many metrics here, potentially useful for model comparrison.

Firstly, this seems to be a tool that is usefult for tuning, modifying, or comparing models for a specific task. The scores themselves might not be entirely meaningful as they may vary depending on the task, but given the same task they allow us to make comparissons.

This is useful for hyperparameter tuning, for example, how we clean the text. We can measure performance with one method, and compare against another..

Whilst I believe that a careful application and interpretation of these test may lead to approximately good decisions being made about model choices, there are many pitfalls.

---

---

## $\color{red}{Process}$

---

It is important to understand that this is a holistic RAG measurement system. To make assessment it relies on questions, context, ground truth answers, and system answers.

Given that we only usually have the context, everything else will probably need to be generated.

* Questions - given a chunck of text, use a good LLM to reverse engineer a question that the context can answer.

* Context - The text chuncks made from your documents.

* Ground Truth Answers - given the questions, and the contexts, get the best available model to generate a reference response.

* System answers - These are the responses that are generated by your actual LLM, the one linked to your RAG chain.

---

---


## $\color{red}{Metrics}$

---

### $\color{green}{Context\hspace{0.3cm}Precision}$

---

This is a measure of how relevant our list of recomendations is. The highest possible score is 1, when every item on our recommendation list up to k, is relavent. The score decreases when the recommendation list is inpure, ie. It contains items that are not relavent. If the non-relavent items are towards the top of the list, we are punished more.

$Precision@K = \frac{TP@k}{TP@k + FP@k}$

This is fairly easy to understand, whenever we measure precision @ k, we use the standard definition, for a list of size k, each relavent document is a TP, and each non-relavent document is a FP.

Now for the full statistic.

$ContextPrecision@k = \frac{\sum_{k=1}^K Precision@k \cdot \mathbb{I}_k} {total-relavent-docs @ k}$

Here for each value up to K, we add the precision at k if the document is relavent. The indicator function indicates if the document is relevant.

In essence we are measuring the purity of the recommendation list, penalising the model more for mistakes made earlier in the list...

[[Relavent], [Non-Relavent], [Relavent]]

$\frac{1 + \frac{2}{3}}{2} = 0.833$

[[Non-Relavent], [Relavent], [Relavent]]

$\frac{\frac{1}{2} + \frac{2}{3}}{2} = 0.417$

But note that...

[[Relavent], [Relavent], [Non-Relavent]]

$\frac{1 + \frac{2}{2}}{2} = 1$

[[Relavent], [Relavent], [Relavent]]

$\frac{1 + \frac{2}{2} + \frac{3}{3}}{3} = 1$

So we have a useful, but not perfect measure of recommendation  purity.

---

---

### $\color{green}{Context\hspace{0.3cm}Relevancy}$

---

This is simply defined as $\frac{|S|}{Total-retreived-sentences}$

Where $|S|$ is the cardinality of the set of sentences that are useful for answering the query.

Here we imagine a system that used parent documents to catch all might get good scores on context precision, because the predictions are good. But it does well by providing lots of information to the LLM, even if some of this is redundant. The context relevancy is a calculation the measures the ratio  of in context information that is useful.

---

---

### $\color{green}{Context \hspace{0.3cm} Recall}$

---

RAGAS context precision is a question purely about the relavency of the predictions made by the embedding model. The context recall is more of a test of the LLM.

Fundamentally It takes a ground truth answer and splits it into sentences. Out of all of these sentences, how many of them can be attributed to in-context information.

$\frac{GT-attributed-to-recommendations}{GT-total-sentneces}$

Usually we think of Recall as measuring False negatives. The recomendation is missing real relevant documents. Tis metric doesn't really measure that, we can concoct a situation where all relevant documents are fetched, but if the model starts to invent things, or over elaborate, then the context recall will go down.

---

---

### $\color{green}{Context \hspace{0.3cm} Entities \hspace{0.3cm} Recall}$

---

$\frac{|CE \cap GE|}{GE}$

Here GE is the set of entities in the ground truth, and CE is the set of entities in the Context. So we're getting the ratio of the intersection of entities, to the total entities. Where the definition of entities is a little opaque.

Essentially we know the real answer talks along certain themes. Is the embedding model returning documents along those themes.

---

---

### $\color{green}{Answer \hspace{0.3cm} Relevancy}$

---

$\frac{1}{N} \sum_{i=}^N cos(E_{gi}, E_0)$

Where $E_{gi}$ is a questions that has been reverse engineered from the system answer (Ask an LLM, what question gave this answer), and $E_0$ is the original question.

Measuring the cosine similarity should give one if we can reverse engineer the orinial question from the response. Of course we are still at the merci of the LLM generating the question from the answer, which is a source of error.

---

---

### $\color{green}{Answer \hspace{0.3cm} Semantic \hspace{0.3cm} Similarity}$

---

This measures similarity from the system response and some ground truth response. This ground truth response is probably a powerfull LLM. There may be variation between, with the length and style of response. If the ground truth really is a human annotated ideal response, then this metric is an assessment about whether we are getting what we want.

---

---

### $\color{green}{Answer \hspace{0.3cm} Correctness}$

---

This is a weighted average (the weights are a parameter), between Answer Semantic Similarity and factual correctness. Factual correctness decomposes the ground truth and the system answer into facts, then the F1 is calculated.

$F1 = \frac{TP}{TP + 0.5(FP + FN)}$

Again we have to rely on some system that will extract the facts from our answers, and this may be partially erronous.

---

---

### $\color{green}{Faithfulness}$

---

This is an assessment of how well the answer adheres to what appears in the context. So it is an assessment of the LLM's ability to stick to what is said in the context.

Crucially the system answer is decomposed into statements / claims.

$\frac{count-claims-inferred-from-context}{total-count-claims-in-response}$

---

---

### $\color{green}{Aspect \hspace{0.3cm} Critique}$

---

This is a metric essentially to address the ethical side of the response. It is not explained here, but it attempts to quantify "harmfulness, maliciousness, coherence, correctness, and conciseness"


## $\color{red}{Potential\hspace{0.3cm}Problems}$  

---

* The calculation of each metric must be taken in account:

Whilst this seems obvious, certain metrics may be of little interest under the circumstances, and if this is the case, we should not attach too much weight. For instance, Context Relevancy measures the proportion of the retrieved context that is useful for answering the question. We may have used a strategy "small to big" where we supply lots of context arround the each retrieved chunck. This will decrease context relevancy, but it may still be useful for the LLM to understand the linguistic context of the chunck.

* Different cuts affect the metrics

Splitting 1000000 token into chuncks of size 500 gives 2000 chuncks, splitting into chunks of size 200 gives 5000 chunks. Smaller chunks lead to more accurate embeddings, but it is harder to pick the correct answer from a large number of chuncks than a smaller number of chunks. Essentially this means it is difficult to compare metrics when the dataset has changed. The change might be due to the difference in the measurement and not from the efficiency of the method. When this is the case we should look to downstream metrics, measuring the overall performance of the system.

* Measurement Often relies on LLMs and there will be some error due to the system of measurement and not from the system being tested.
  * Ground truth questions are generated from the docs (probably by an LLM)
  * Ground truth answers are generated from the docs and the questions (probably by an llm)
  * Some metrics require reverse engineering, or decomposition into statements (tasks performed by an LLM)

All of these technique are prone to some amount of error. In addition we want to use the most powerful model available for this work, to get the most accurate model possible.

* Testing is heavy on token generation.

When you consider that we require questions, answers, decompositions, and fact checking. We are generating lots of tokens for each test using expensive models. Whilst a given open source LLM and embedding model will only cost GPU usage, testing is likely to used paid models, and will be relatively expensive.

* Contradictory Metrics

It is possible faithfulness increases, meaning the LLM is sticking better to the info in the context. This might be at the detriment of the Answer Relevancy because the LLM is now failing to answer the question correctly. Whilst we talk about finetuning and other big strategies, a simple change to the LLM prompt like "use the context info and common sense" may have a big impact on the results.

* Interdependancy

Increasing answer relevency might seem like an issue with the LLM, but imporvement of the embedding model might lead to a downstream improvement that ends up in increased answer relevancy. Generally there is no way of knowing where the system might be best improved, experience and intuition will play a big part because of the exponential nature of the search problem.

* Ambiguous Mapping between Performance and Metrics

The performance of the system is to a large degree subjective. What is a good answer? No set of metrics is likely to be able to fully explain what we mean by "good" for a given problem.

---

---

## $\color{red}{Positives}$

---

There are so many modifications that can be made throughout the RAG pipeline. The subleties in response variation are difficult to measure. Some metrics eg. Context Similarity (based on the popular MAP@k) are inherently useful. Others eg. Faithfulness, could be useful in the right scenario. But across the 9 metrics, there is enough to act as a guide and allow us to heuristically improve the system. I say heuristically because we cannot test everything, we might look to choose between two embedding models A and B, having chosen B we determine an optiamal context length for model B, unaware that a modification to context length for model A would have given the best results. If we consider 8 different decisions of system design, and 3 options for each decision, we need $3^8 = 6561$ tests. Obvious some decision will need to be made in isolation in spite of obvious dependence.

---

---


---

## $\color{red}{Conclusions}$

---

Do: Use this tool to compare RAG systems.

Don't: Interpret any metric without careful consideration of its meaning.

Do: Think very carefully about what you want to improve, and how you can do it.

Don't: Discount other potentially important measurement techniques like MRR.




----

---

---

---

## $\color{blue}{Workflow}$

The workflow for evaluation has a number of steps. Essentially RAGAS will do everything in the background to analyse the models provided we supply it with a RAGAS dataset.

The RAGAS dataset is simply a list of dictionaries:

* question
* answer
* contexts
* ground truths

The problem is that we start with some text data, which is the contexts that we will be retrieving from a vector store. The questions, answers (our RAG response) and the ground truth responses do not exist. We will need to generate these.

The final workflow looks like this:

* Load the documents

* Bag the documents

* Make a retriever

* Create a prompt template

* Create a QA chain

* Create the Question Generator Prompt

* Create the Question generator chain

* Generate dataset questions

* Generate dataset ground truth

* Create a Transformers Dataset from pandas

* Import the metrics from RAGAS

* Add the system answers to the RAGAS dataset- list of dictionaries:
  * [{'question', 'answer', 'contexts', 'ground_truths'}, ...]

* Make a QA factory to build data for other models


---
## $\color{blue}{Load Docs}$
---

To give us an interesting subject matter we will get the English Theft Act 1968. The language for should provide a challenge for our embedding models.



Once downloaded, the [Theft Act 1968](https://www.legislation.gov.uk/ukpga/1968/60/pdfs/ukpga_19680060_en.pdf) needs to be split into Langchain Documents. Examining the document we a specific structure.

"(4) The Police (Property) Act 1897 (which makes provision 1897 c. 30.
for the disposal of property in the possession of the police) shall
apply to property which has come into the possession of the
police under this section as it applies to property which has
come into the possession of the police in the circumstances
mentioned in that Act.

(5) This section is to be construed in accordance with section
24 of this Act ; and in subsection (2) above the references to
handling stolen goods shall include any corresponding offence
committed before the commencement of this Act"

Intuitively we want to try and split the text into paragraphs.

In [ ]:
%%capture
!pip install -U -q langchain ragas pymupdf

In [ ]:
# Get a pdf reader
from langchain_community.document_loaders import PyMuPDFLoader

In [ ]:
# Link to file
from google.colab import drive

drive.mount("/content/drive")
%cd '/content/drive/MyDrive/'

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
# Get pdf from repo
data = PyMuPDFLoader("RAG_tutorial/Data/theft_act_1968.pdf").load()


In [ ]:
data

[Document(page_content='Theft Act 1968 \nCHAPTER 60 \nARRANGEMENT OF SECTIONS \nDefinition of "theft" \nSection \n1. \nBasic definition of theft. \n2. " Dishonestly ". \n3. " Appropriates ". \n4. " Property ". \n5. " Belonging to another ". \n6. " \nWith the intention of permanently depriving the other \nof it". \nTheft, robbery, burglary, etc. \n7. Theft. \n8. \nRobbery. \n9. \nBurglary. \n10. \nAggravated burglary. \n11. \nRemoval of articles from places open to the public. \n12. Taking motor vehicle or other conveyance without authority. \n13. \nAbstracting of electricity. \n14. \nExtension to thefts from mails outside England and Wales, \nand robbery etc. on such a theft. \nFraud and blackmail \n15. \nObtaining property by deception. \n16. Obtaining pecuniary advantage by deception. \n17. \nFalse accounting. \n18. Liability of \ncompany officers for certain offences by company. \n19. \nFalse statements by company directors, etc. \n20. \nSuppression, etc. of documents. \n21. \nBlack

In [ ]:
type(data[0])

langchain_core.documents.base.Document

In [ ]:
data[0]

Document(page_content='Theft Act 1968 \nCHAPTER 60 \nARRANGEMENT OF SECTIONS \nDefinition of "theft" \nSection \n1. \nBasic definition of theft. \n2. " Dishonestly ". \n3. " Appropriates ". \n4. " Property ". \n5. " Belonging to another ". \n6. " \nWith the intention of permanently depriving the other \nof it". \nTheft, robbery, burglary, etc. \n7. Theft. \n8. \nRobbery. \n9. \nBurglary. \n10. \nAggravated burglary. \n11. \nRemoval of articles from places open to the public. \n12. Taking motor vehicle or other conveyance without authority. \n13. \nAbstracting of electricity. \n14. \nExtension to thefts from mails outside England and Wales, \nand robbery etc. on such a theft. \nFraud and blackmail \n15. \nObtaining property by deception. \n16. Obtaining pecuniary advantage by deception. \n17. \nFalse accounting. \n18. Liability of \ncompany officers for certain offences by company. \n19. \nFalse statements by company directors, etc. \n20. \nSuppression, etc. of documents. \n21. \nBlackm

In [ ]:
%%capture
%pip install -qU langchain-text-splitters

In [ ]:
from langchain.text_splitter import CharacterTextSplitter


In [ ]:
# split the text
text_splitter = CharacterTextSplitter(
    separator= r"\n.?\d",
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=True,
)

texts = text_splitter.transform_documents(data)

In [ ]:
len(texts)


260

In [ ]:
for i in range(12,18):
  print()
  print(i)
  print(texts[i].page_content)


12
68 CHAPTER 60 
An Act to revise the law of England and Wales as to theft 
and similar or associated offences, and in connection 
therewith to make provision as to criminal proceedings 
by one party to a marriage against the other, and to 
make certain amendments extending beyond England 
and Wales in the Post Office Act 1953 and other 
enactments; and for other purposes connected there- 
with.

13
6th July 1968] 
B 
E IT ENACTED by the Queen's most Excellent Majesty, by and 
with the advice and consent of the Lords Spiritual and 
Temporal, and Commons, in this present Parliament 
assembled, and by the authority of the same, as follows 
Definition of " theft "

14
.-(1) A person is guilty of theft if he dishonestly appro- Basic 
priates property belonging to another with the intention of definition 
permanently depriving the other of it ; and " thief " and " steal " of theft. 
shall be construed accordingly.

15
) It is immaterial whether the appropriation is made with a 
view to ga

---
## $\color{blue}{Bag-Docs}$
---

We want to bag our documents, upload them into a vector store. The document splitting has been complicated.
* There is text in the margins, interfering with the main text.
* We get some chunks which are part of a larger paragraph and have no inherent meaning on there own.

For the first problem, we could look to get another source of data. Scraping from HTML would be the best solution because we can add all the sections in the metadata. But we won't concern ourselves with this at the moment.

For the second problem, we could go further than our regex splitting but a quick fix would be to include the surrounding context with the text. This is small to big, we embed smaller chunck, but return the target chunk and surrounding chunks to the LLM. This way we hope that all context will be coherent.

In [ ]:
%%capture
pip install langchain-community

---
##### $\color{red}{Embedding-Model}$
---

In [ ]:
%%capture
!pip install sentence_transformers

In [ ]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
small_embeddings = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


---
##### $\color{red}{Vector-Store}$
---

In [ ]:
%%capture
!pip install chromadb

In [ ]:
from langchain_community.vectorstores.chroma import Chroma

In [ ]:
persist_directory = "RAG_tutorial/dbs/theft_act.db"

In [ ]:
# db = Chroma.from_documents(
#     collection_name= 'v_db',
#     persist_directory=persist_directory,
#     embedding=embeddings,
#     documents=texts)

In [ ]:
small_persist_directory = "RAG_tutorial/dbs/small_theft_act.db"

In [ ]:
# small_db = Chroma.from_documents(
#     collection_name= 'small_v_db',
#     persist_directory=small_persist_directory,
#     embedding=small_embeddings,
#     documents=texts)

In [ ]:
vs = Chroma(
    collection_name= 'smallv_db',
    persist_directory=persist_directory,
    embedding_function=embeddings,
)

In [ ]:
small_vs = Chroma(
    collection_name= 'small_v_db',
    persist_directory=small_persist_directory,
    embedding_function=small_embeddings,
)

In [ ]:
query = "Is it ok to take a wild animal?"
res = vs.similarity_search(query, k=2)

In [ ]:
for doc in res:
  print('\n-------------------\n')
  print(doc.page_content)


-------------------

) Wild creatures, tamed or untamed, shall be regarded as 
property ; but a person cannot steal a wild creature not tamed 
nor ordinarily kept in captivity, or the carcase of any such 
creature, unless either it has been reduced into possession by or 
on behalf of another person and possession of it has not since 
been lost or abandoned, or another person is in course of reduc- 
ing it into possession.

-------------------

.-(1) A person who unlawfully takes or kills, or attempts to take 
or kill, any deer in inclosed land where deer are usually kept shall 
on summary conviction be liable to a fine not exceeding fifty 
pounds or, for an offence committed after a previous conviction 
of an offence under this paragraph, to imprisonment for a term not 
exceeding three months or to a fine not exceeding one hundred 
pounds or to both.


In [ ]:
retriever = vs.as_retriever()

In [ ]:
small_retriever = small_vs.as_retriever()

---
## $\color{blue}{LLM}$
---

In [ ]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API Key: ··········


In [ ]:
%%capture
!pip install langchain_openai

In [ ]:
from langchain_openai import ChatOpenAI
qa_llm = ChatOpenAI(model="gpt-4o", temperature = 0)

---
## $\color{blue}{Make-Questions}$
---

Now we need to get the questions from our initial documents.

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="A question about the context."
)

question_response_schemas = [
    question_schema,
]

In [ ]:
question_schema.__dict__

{'name': 'question',
 'description': 'A question about the context.',
 'type': 'string'}

In [ ]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()

In [ ]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"question": string  // A question about the context.\n}\n```'

In [ ]:
qa_template = """\
You are a University Professor creating a test for advanced students. For each context, create a question that is specific to the context. Avoid creating generic or general questions.

question: a question about the context.

{format_instructions}

context: {context}
"""

In [ ]:
qa_template = ChatPromptTemplate.from_template(template=qa_template)

In [ ]:
print(qa_template.format(context="THis is a CONTEXT context baby", format_instructions=format_instructions))

Human: You are a University Professor creating a test for advanced students. For each context, create a question that is specific to the context. Avoid creating generic or general questions.

question: a question about the context.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // An answer to the question
}
```

context: THis is a CONTEXT context baby



In [ ]:
bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

In [ ]:
bare_template.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'content': {'title': 'Content', 'type': 'string'}}}

In [ ]:
messages = qa_template.format_messages(
    context=texts[83],
    format_instructions=format_instructions
)

In [ ]:
messages

[HumanMessage(content='You are a University Professor creating a test for advanced students. For each context, create a question that is specific to the context. Avoid creating generic or general questions.\n\nquestion: a question about the context.\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // An answer to the question\n}\n```\n\ncontext: page_content=\') A person guilty of blackmail shall on conviction on indict- \\nment be liable to imprisonment for a term not exceeding \\nfourteen years. \\nOffences relating to goods stolen etc. \\nHandling\' metadata={\'source\': \'RAG_tutorial/Data/theft_act_1968.pdf\', \'file_path\': \'RAG_tutorial/Data/theft_act_1968.pdf\', \'page\': 11, \'total_pages\': 37, \'format\': \'PDF 1.4\', \'title\': \'\', \'author\': \'\', \'subject\': \'\', \'keywords\': \'\', \'creator\': \'PdfCompressor 3.1.41\', \'producer\': \'CVISIO

In [ ]:
print(messages[0].content)

You are a University Professor creating a test for advanced students. For each context, create a question that is specific to the context. Avoid creating generic or general questions.

question: a question about the context.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // An answer to the question
}
```

context: page_content=') A person guilty of blackmail shall on conviction on indict- \nment be liable to imprisonment for a term not exceeding \nfourteen years. \nOffences relating to goods stolen etc. \nHandling' metadata={'source': 'RAG_tutorial/Data/theft_act_1968.pdf', 'file_path': 'RAG_tutorial/Data/theft_act_1968.pdf', 'page': 11, 'total_pages': 37, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PdfCompressor 3.1.41', 'producer': 'CVISION Technologies', 'creationDate': "D:20071101124339Z00'00'", 'modDate': '', 'trapp

In [ ]:
question_generation_chain = bare_template | qa_llm

In [ ]:
response = question_generation_chain.invoke({"content" : messages})
output_dict = question_output_parser.parse(response.content)

In [ ]:
response

AIMessage(content='```json\n{\n\t"question": "According to the Theft Act 1968, what is the maximum term of imprisonment for a person convicted of blackmail on indictment?",\n\t"answer": "Fourteen years"\n}\n```', response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 316, 'total_tokens': 362}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-db9b7424-4cd3-4c29-a176-94f41260e126-0', usage_metadata={'input_tokens': 316, 'output_tokens': 46, 'total_tokens': 362})

In [ ]:
output_dict

{'question': 'According to the Theft Act 1968, what is the maximum term of imprisonment for a person convicted of blackmail on indictment?',
 'answer': 'Fourteen years'}

---
##### $\color{red}{Loop-Docs}$
---

Simply loop the docs and get GPT 4o to generate likely questions.

In [ ]:
%%capture
!pip install -qU tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
qac_triplets = []

for text in tqdm(texts[60:80]):

  # Gernerate a question from the text
  messages = qa_template.format_messages(
    context=text,
    format_instructions=format_instructions
  )
  response = question_generation_chain.invoke({"content" : messages})

  try:
    output_dict = question_output_parser.parse(response.content)
  except Exception as e:
    continue

  #make a dict out of question and context and append it to triplets
  output_dict['context'] = text
  qac_triplets.append(output_dict)


100%|██████████| 20/20 [00:52<00:00,  2.61s/it]


In [ ]:
qac_triplets[0]

{'question': 'According to subsection (1) of the document, which specific jurisdictions are included in the British postal area?',
 'answer': 'The several jurisdictions of England and Wales, of Scotland, of Northern Ireland, of the Isle of Man, and of the Channel Islands.',
 'context': Document(page_content=') In subsection (1) above the reference to different jurisdic- \ntions in the British postal area is to be construed as referring \nto the several jurisdictions of England and Wales, of Scotland, \nof Northern Ireland, of the Isle of Man and of the Channel \nIslands.', metadata={'source': 'RAG_tutorial/Data/theft_act_1968.pdf', 'file_path': 'RAG_tutorial/Data/theft_act_1968.pdf', 'page': 8, 'total_pages': 37, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PdfCompressor 3.1.41', 'producer': 'CVISION Technologies', 'creationDate': "D:20071101124339Z00'00'", 'modDate': '', 'trapped': ''})}

---
## $\color{blue}{Make-Answers}$
---

Now we have an original doc, a question made from the doc, we need to generate a ground truth answer.

In [ ]:
answer_schema = ResponseSchema(
    name="answer",
    description="An answer to the question"
)

In [ ]:
answer_response_schemas = [
    answer_schema,
]

In [ ]:
answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)

In [ ]:
format_instructions = answer_output_parser.get_format_instructions()

In [ ]:
answer_template = """\
You are a University Professor creating a test for advanced students. For each question and context, create an answer.

answer: a answer about the context.

Format the output as JSON with the following keys:
answer

question: {question}
context: {context}
"""

answer_template = ChatPromptTemplate.from_template(template=answer_template)

In [ ]:
messages = answer_template.format_messages(
    context="The only real pizza flavour is mozarella and tomoatoes, nothing else can be considered a pizza.",
    question="Is a Regina a good pizza flavor?",
    format_instructions=format_instructions
)

In [ ]:
messages

[HumanMessage(content='You are a University Professor creating a test for advanced students. For each question and context, create an answer.\n\nanswer: a answer about the context.\n\nFormat the output as JSON with the following keys:\nanswer\n\nquestion: Is a Regina a good pizza flavor?\ncontext: The only real pizza flavour is mozarella and tomoatoes, nothing else can be considered a pizza.\n')]

In [ ]:
answer_generation_chain = bare_template | qa_llm

In [ ]:
response = answer_generation_chain.invoke({"content" : messages})
output_dict = answer_output_parser.parse(response.content)

In [ ]:
output_dict

---
##### $\color{red}{Loop-Docs+Questions}$
---

Now we simply loop the docs and questions, and get the ground truth. We know this ground truth has the context that relates to the question, and the powerful model gives us a theoretical best response.

In [ ]:
for doublet in tqdm(qac_triplets):
  messages = answer_template.format_messages(
      context=doublet['context'],
      question=doublet['question'],
      format_instructions=format_instructions
  )
  response = answer_generation_chain.invoke({'content': messages})
  try:
    output_dict = answer_output_parser.parse(response.content)
  except Exception as e:
    continue
  doublet['answer'] = output_dict['answer']

100%|██████████| 12/12 [00:50<00:00,  4.22s/it]


In [ ]:
import pandas as pd
from datasets import Dataset

ground_truth_qac_set = pd.DataFrame(qac_triplets)
ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "ground_truth"})

eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [ ]:
# load up an old version
from datasets import load_dataset
eval_dataset = load_dataset("RAG_tutorial/res/eval_dataset")

In [ ]:
ground_truth_qac_set.context

0     ) In subsection (1) above the reference to dif...
1     8 \nObtaining \npecuniary \nadvantage \nby \nd...
2     ) For purposes of this section "deception" mea...
3     .-(1) A person who by any deception dishonestl...
4     ) The cases in which a pecuniary advantage wit...
5     .-(1) Where a person dishonestly, with a view ...
6     ) For purposes of this section a person who ma...
7     .-(1) Where an offence committed by a body cor...
8     ) Where the affairs of a body corporate are ma...
9     ) For purposes of this section a person who ha...
10    ) Where the affairs of a body corporate or ass...
11    .-(1) A person who dishonestly, with a view to...
Name: context, dtype: object

In [ ]:
eval_dataset

Dataset({
    features: ['question', 'ground_truth', 'context'],
    num_rows: 12
})

In [ ]:
eval_dataset['question']

['According to subsection (1) of the document, which specific jurisdictions are included in the British postal area?',
 'How does Section 6 of the Theft Act 1968 apply to the context of obtaining pecuniary advantage by deception, as mentioned on page 9 of the document?',
 "According to the definition provided in the Theft Act 1968, what constitutes 'deception' and what are the possible forms it can take?",
 'According to the excerpt from the Theft Act 1968, what is the maximum term of imprisonment for a person who dishonestly obtains a pecuniary advantage by deception?',
 'According to the provided section of the Theft Act 1968, what are the specific scenarios in which a pecuniary advantage is considered to be obtained for a person?',
 'According to the provided excerpt from the Theft Act 1968, what are the specific actions that can lead to a person being liable to imprisonment for a term not exceeding seven years?',
 'According to the Theft Act 1968, what legal implications might aris

---
## $\color{blue}{Test-Chains}$
---


In [ ]:
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

template = """Answer the question by referring to information in the context:

### CONTEXT
{context}

### QUESTION
Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
def create_qa_chain(retriever, llm):

  input_chain = RunnableParallel(context=retriever, input = RunnablePassthrough())
  output_chain = RunnableParallel(context = itemgetter('context'), answer =  prompt | llm)

  return input_chain | output_chain

In [ ]:
llm_3 = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_4 = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
chain_3_big = create_qa_chain(retriever,llm_3)
chain_4_small = create_qa_chain(small_retriever, llm_4)

In [ ]:
res = chain_4_small.invoke("Can we eat wild animals")

In [ ]:
res['answer'].content

'The provided context does not directly address whether wild animals can be eaten. It primarily discusses the legal aspects of taking or killing wild animals, particularly in relation to property and theft laws. For example, it mentions that wild creatures, whether tamed or untamed, are regarded as property and outlines the legal consequences of unlawfully taking or killing deer in enclosed land.\n\nTo determine whether wild animals can be eaten, you would need to refer to specific wildlife conservation laws, hunting regulations, and food safety laws that govern the consumption of wild animals in your jurisdiction.'

make the ragas dataset

In [ ]:
eval_dataset

Dataset({
    features: ['question', 'ground_truth', 'context'],
    num_rows: 12
})

In [ ]:
def create_ragas_dataset(chain, dataset):
  rag_dataset = []
  questions = dataset['question']
  ground_truths = dataset['ground_truth']
  for i in tqdm(range(len(dataset))):
    answer = chain.invoke(questions[i])

    rag_dataset.append(
        {"question" : questions[i],
         "answer" : answer["answer"].content,
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : ground_truths[i]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

In [ ]:
# ragas_dataset_3_big = create_ragas_dataset(chain_3_big, eval_dataset)

100%|██████████| 12/12 [00:30<00:00,  2.58s/it]


In [ ]:
# ragas_dataset_3_big = ragas_dataset_3_big.rename_column("ground_truths", "ground_truth")

In [ ]:
ragas_dataset_3_big = load_dataset("RAG_tutorial/res/dataset_3_big")

In [ ]:
# ragas_dataset_4_small = create_ragas_dataset(chain_4_small, eval_dataset)

100%|██████████| 12/12 [00:27<00:00,  2.28s/it]


In [ ]:
# ragas_dataset_4_small = ragas_dataset_4_small.rename_column("ground_truths", "ground_truth")

In [ ]:
ragas_dataset_4_small = load_dataset("RAG_tutorial/res/dataset_3_big")

In [ ]:
type(ragas_dataset_3_big)

datasets.arrow_dataset.Dataset

---
## $\color{blue}{Evaluate-Ragus}$
---


In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)

from ragas.metrics.critique import harmfulness
from ragas import evaluate

def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
  )
  return result

In [ ]:
# llm_3_big = evaluate_ragas_dataset(ragas_dataset_3_big)

Evaluating:   0%|          | 0/84 [00:00<?, ?it/s]

In [ ]:
type(llm_3_big)

ragas.evaluation.Result

In [ ]:
res_3 = llm_3_big.to_pandas().iloc[:,4:]

In [ ]:
res_3 = pd.read_pickle("RAG_tutorial/res/llm_3_big")

In [ ]:
# llm_4_small = evaluate_ragas_dataset(ragas_dataset_4_small)

Evaluating:   0%|          | 0/84 [00:00<?, ?it/s]

In [ ]:
res_4 = llm_4_small.to_pandas().iloc[:,4:]

In [ ]:
res_4 = pd.read_pickle("RAG_tutorial/res/llm_4_small")

In [ ]:
res_3.mean(axis=0)

context_precision     0.962963
faithfulness          0.888889
answer_relevancy      0.967991
context_recall        0.958333
context_relevancy     0.050082
answer_correctness    0.728228
answer_similarity     0.969467
dtype: float64

In [ ]:
res_4.mean(axis=0)

context_precision     0.956019
faithfulness          0.958333
answer_relevancy      0.965989
context_recall        0.791667
context_relevancy     0.063085
answer_correctness    0.604390
answer_similarity     0.978004
dtype: float64

In [ ]:
llm_4_small.to_pandas().to_pickle("RAG_tutorial/res/llm_4_small")
llm_3_big.to_pandas().to_pickle("RAG_tutorial/res/llm_3_big")

In [ ]:
ragas_dataset_3_big.save_to_disk("RAG_tutorial/res/dataset_3_big")

Saving the dataset (0/1 shards):   0%|          | 0/12 [00:00<?, ? examples/s]

In [ ]:
ragas_dataset_4_small.save_to_disk("RAG_tutorial/res/dataset_4_small")

Saving the dataset (0/1 shards):   0%|          | 0/12 [00:00<?, ? examples/s]

In [ ]:
eval_dataset.save_to_disk("RAG_tutorial/res/eval_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/12 [00:00<?, ? examples/s]